In [1]:
import numpy as np
import helper

In [2]:
# FashionMNIST0.3.npz
Xtr, Str, Xts, Yts = helper.load_and_inspect("datasets/FashionMNIST0.3.npz")

Dataset loaded with the following shapes:
Xtr: (18000, 784)
Str: (18000,)
Xts: (3000, 784)
Yts: (18000, 784)
